In [ ]:
import rdflib
from rdflib import RDF, RDFS, XSD, OWL
from rdflib.paths import OneOrMore

import re
import pandas as pd

from bluegraph import PandasPGFrame
from bluegraph.backends.neo4j import pgframe_to_neo4j 

In [ ]:
from bmo_tools.neo4j import ontology_to_neo4j, execute
from bmo_tools.ontologies import subontology_from_term

## 1. Import BMO

In [ ]:
uri = "bolt://127.0.0.1:7687"
username = "neo4j"
password = "admin"

In [ ]:
ontology_to_neo4j(uri, username, password, "../../ontologies/bmo.ttl", format="turtle")

## 2. Extract UBERON subontology (top down from 'regional part of brain')

In [ ]:
g = rdflib.Graph()
g.parse("../../ontologies/uberon.ttl", format="turtle")

Find the term of interest

In [ ]:
label = "regional part of brain"
term_to_expand = ""
for s in g.subjects(RDFS.label, rdflib.Literal(label, datatype=XSD.string)):
    term_to_expand = s
    break

In [ ]:
term_to_expand

In [ ]:
subontology = subontology_from_term(g, term_to_expand, closed=True)

## 3. Create a PGFrame and push it into Neo4j

In [ ]:
def remove_uri(p):
    match = re.match("(http:\/\/.*)#(.*)", p)
    if match:
        return match.groups()[1]
    else:
        return p.split("/")[-1]

In [ ]:
frame = PandasPGFrame.from_ontology(rdf_graph=subontology, format="turtle", remove_prop_uris=True)
frame._nodes["@type"] = "UBERON_CLASS"
frame._nodes = frame._nodes.rename(columns={"id": "uberon_id"})
frame.rename_node_properties({
    p: p.replace(" ", "_") for p in frame.node_properties()})
for p in frame.node_properties():
    frame.node_prop_as_category(p)
frame._edges["@type"] = frame._edges["@type"].apply(
    lambda x: {remove_uri(el.replace(" ", "_").replace("(", "_").replace(")", "_")) for el in x})
frame._edge_prop_types["@type"] = "category"

In [ ]:
neo4j_graph = pgframe_to_neo4j(
    frame, uri=uri,
    username=username,
    password=password, 
    node_label="ONTOLOGY_CLASS", 
    node_types_as_labels=True,
    edge_types_as_labels=True)

In [ ]:
query = "MATCH (n {id: 'Brain Region'}), (m {id: 'regional part of brain'}) MERGE (n)-[:EQUIVALENT_CLASS]->(m)"
execute(neo4j_graph.driver, query)

## 4. Read Allen MBA  CCFv3 Ontology

In [ ]:
mba = rdflib.Graph()
mba.parse("../../ontologies/allen_MBA_ontology_ccfv3.ttl", format="turtle")

In [ ]:
frame._nodes.loc[frame._nodes["database_cross_reference"].notna(), "MBA_ref"] = frame._nodes["database_cross_reference"][frame._nodes["database_cross_reference"].notna()].apply(
    lambda x: [el for el in x if el.startswith("MBA")])
frame._nodes["MBA_ref"] = frame._nodes["MBA_ref"].apply(
    lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)

In [ ]:
mba_indices = frame._nodes[frame._nodes["MBA_ref"].notna()]["MBA_ref"].tolist()

In [ ]:
allen_terms = {}
for s in mba.subjects(RDFS.subClassOf, rdflib.URIRef("https://neuroshapes.org/BrainRegion")):
    for o in mba.objects(s, rdflib.URIRef("http://schema.org/identifier")):
#         if f"MBA:{o.value}" in mba_indices:
        allen_terms[f"MBA:{o.value}"] = s

In [ ]:
selected_mba = rdflib.Graph()
for t in allen_terms.values():
    subontology = subontology_from_term(
        mba, t, closed=False)
    selected_mba += subontology

In [ ]:
mbaframe = PandasPGFrame.from_ontology(rdf_graph=selected_mba, remove_prop_uris=True)
mbaframe.rename_node_properties({
    n: remove_uri(n) for n in mbaframe.node_properties()
})
mbaframe._nodes["@type"] = "ALLEN_MBA_CLASS"
mbaframe._edges["@type"] = mbaframe._edges["@type"].apply(lambda x: {remove_uri(el) for el in x})
mbaframe.rename_nodes({
    n: f"Mouse {n}" for n in mbaframe.nodes()
})
mbaframe.node_prop_as_numeric("st_level")
mbaframe.node_prop_as_numeric("hemisphere_id")
mbaframe.node_prop_as_numeric("graph_order")
# mbaframe.node_prop_as_numeric("atlas_id")

In [ ]:
pgframe_to_neo4j(
    mbaframe, uri=uri, username=username, password=password, 
    node_label="ONTOLOGY_CLASS", 
    node_types_as_labels=True,
    edge_types_as_labels=True)

In [ ]:
query = """
MATCH (n:ALLEN_MBA_CLASS), (m:UBERON_CLASS)
WHERE "MBA:" + n.identifier in m.database_cross_reference
MERGE (n)-[:IS_SPECIALIZATION_OF]->(m)
"""
execute(neo4j_graph.driver, query)

In [ ]:
query = """
MATCH (n:ALLEN_MBA_CLASS), (m {id: "Mus musculus"})
MERGE (n)-[:species]->(m)
"""
execute(neo4j_graph.driver, query)